<a href="https://colab.research.google.com/github/the-cmyk/SandBox/blob/master/RealInsightfacePrune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
pip install git+https://github.com/nizhib/pytorch-insightface

  Cloning https://github.com/nizhib/pytorch-insightface to /tmp/pip-req-build-56yiddqr
  Running command git clone -q https://github.com/nizhib/pytorch-insightface /tmp/pip-req-build-56yiddqr
  Created wheel for insightface: filename=insightface-0.1.0+8596a0b-cp36-none-any.whl size=4271 sha256=f48084c6a2833edd53aa143782062814559c24f896c560ac822a67ee8d7a8bb6
  Stored in directory: /tmp/pip-ephem-wheel-cache-sfqxrvl7/wheels/10/e7/5a/648c05950e72cdf4d184d44d74e6a94297aa5f9b529f05ad06
Successfully built insightface


In [36]:
!git clone https://github.com/nizhib/pytorch-insightface.git

Cloning into 'pytorch-insightface'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 23 (delta 4), reused 15 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [37]:
!git clone https://github.com/VainF/Torch-Pruning.git

Cloning into 'Torch-Pruning'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (214/214), done.
remote: Total 267 (delta 142), reused 131 (delta 49), pack-reused 0
Receiving objects: 100% (267/267), 304.60 KiB | 13.84 MiB/s, done.
Resolving deltas: 100% (142/142), done.


In [38]:
pip install torch_pruning

In [39]:
%cd /content/pytorch-insightface/insightface/

/content/pytorch-insightface/insightface


In [56]:
import torch_pruning as tp
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn 
import iresnet
import insightface

def prune_model(model):
    model.cpu()
    DG = tp.DependencyGraph().build_dependency( model, torch.randn(1, 3, 112, 112) )
    def prune_conv(conv, pruned_prob):
        weight = conv.weight.detach().cpu().numpy()
        out_channels = weight.shape[0]
        L1_norm = np.sum( np.abs(weight), axis=(1,2,3))
        num_pruned = int(out_channels * pruned_prob)
        prune_index = np.argsort(L1_norm)[:num_pruned].tolist() # remove filters with small L1-Norm
        plan = DG.get_pruning_plan(conv, tp.prune_conv, prune_index)
        plan.exec()
    
    block_prune_probs = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.40, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49]
    blk_id = 0
    for m in model.modules():
        if isinstance( m, iresnet.IBasicBlock ):
            #print("conv")
            prune_conv( m.conv1, block_prune_probs[blk_id] )
            prune_conv( m.conv2, block_prune_probs[blk_id] )
            blk_id+=1
    return model
    
model = iresnet.iresnet100(pretrained=True)
torch.save(model, "before.pt")

dummy = torch.randn(1, 3, 112, 112)

params = sum([np.prod(p.size()) for p in model.parameters()])
print("Number of Parameters before: %.1fM"%(params/1e6))
model.eval()
with torch.no_grad():
  before = (model(dummy))

model2 = prune_model(model)
params = sum([np.prod(p.size()) for p in model2.parameters()])
print("Number of Parameters after: %.1fM"%(params/1e6))

cos = nn.CosineSimilarity(dim=1)
print("--------------")
print ("Cosine Similarity: ")
print(cos(before, model2(dummy)))
torch.save(model2, "after.pt")


Number of Parameters before: 65.2M
Number of Parameters after: 4.2M
--------------
Cosine Similarity: 
tensor([0.0870], grad_fn=<DivBackward0>)
